In [1]:
import os
from pathlib import Path


In [2]:
from aocd.models import Puzzle
from collections import Counter, defaultdict
import numpy as np
import re
from statistics import median, mean
import math

# Day 1

In [16]:
puzzle = Puzzle(2022, 1)

In [36]:
elf_snaks = [sum(map(int, line.split("\n"))) for line in puzzle.input_data.split("\n\n")]

## Part 1 

In [40]:
res_a = max(elf_snaks)
res_a

66186

In [34]:
puzzle.answer_a = res_a

That's the right answer!  You are one gold star closer to collecting enough star fruit. [Continue to Part Two]


## Part 2

In [43]:
res_b = sum(sorted(elf_snaks)[-3:])
res_b

196804

In [44]:
puzzle.answer_b = res_b

That's the right answer!  You are one gold star closer to collecting enough star fruit.You have completed Day 1! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


# Day 2

In [112]:
puzzle = Puzzle(2022, 2)

In [113]:
data = [line.split(" ") for line in puzzle.input_data.split("\n")]

## Part 1 

In [114]:
score_mapping = {k: idx+1 for idx, k in enumerate("ABC")}
letter_mapping = dict(zip("XYZ","ABC"))
def parse_pair(e1, e2, mapped=False):
    if not mapped:
        e2 = letter_mapping[e2]
    s1, s2 = score_mapping[e1], score_mapping[e2]
    
    res = (ord(e1) - ord(e2) + 1) % 3
    
    return res * 3 + s1, s2 + (2 - res) * 3

In [115]:
res_a = sum(parse_pair(*pair)[1] for pair in data)
res_a

14531

In [116]:
puzzle.answer_a = res_a

## Part 2

In [117]:
outcome_mapping = {k: idx-1 for idx, k in enumerate("XYZ")}
def parse_outcome(e1, out_come):
    e2 = chr((ord(e1) - ord("A") + outcome_mapping[out_come]) % 3 + ord("A"))
    return parse_pair(e1, e2, mapped=True)

In [118]:
res_b = sum(parse_outcome(*pair)[1] for pair in data)
res_b

11258

In [119]:
puzzle.answer_b = res_b

# Day 3

In [120]:
puzzle = Puzzle(2021, 3)

In [121]:
data = puzzle.input_data.split("\n")

In [122]:
data

['001110000001',
 '010100101000',
 '101101010010',
 '010111101010',
 '100011100110',
 '110100001011',
 '100010001100',
 '011110100110',
 '110011110000',
 '111010011001',
 '111100100011',
 '110101101010',
 '101001111100',
 '001110101110',
 '011100001110',
 '011000101101',
 '101100001000',
 '001010000111',
 '001100101001',
 '100001000011',
 '010010111011',
 '010101101010',
 '100111100001',
 '001011101000',
 '001110100100',
 '110001001011',
 '001101110111',
 '101100100010',
 '100110001111',
 '010111011111',
 '011111111101',
 '011111110011',
 '111011000001',
 '001000011110',
 '111001100001',
 '101111101100',
 '111000001011',
 '101110010111',
 '001111110001',
 '011110000010',
 '100101011101',
 '111000100010',
 '000110111100',
 '111010111100',
 '001110100011',
 '100111001100',
 '011111100101',
 '000010000110',
 '111111010011',
 '011000010011',
 '010101011011',
 '111110010000',
 '001100110110',
 '001111010100',
 '000100000100',
 '001100001011',
 '010010110010',
 '010010101101',
 '001111010101

In [ ]:
def str_bit_to_b_10(bits):
    return int("".join(bits), 2 )

## Part 1 

In [ ]:
numbers = list(map(lambda t: (t[0][0], t[1][0]), [Counter(bits).most_common() for bits in zip(*data)]))

In [ ]:
[Counter(bits).most_common() for bits in zip(*data)]

In [ ]:
final = []
for number in zip(*numbers):
    final.append(str_bit_to_b_10(number))

In [ ]:
res_a = final[0] * final[1]
res_a

In [ ]:
puzzle.answer_a = res_a

## Part 2

In [ ]:
def parse(data, up=True):
    for i, _ in enumerate(data[0]):
        c = Counter(list(zip(*data))[i])
        v_0, v_1 = c["0"], c["1"]

        if v_0 == v_1:
            bit = up
        else:
            bit = ( v_1 < v_0 ) ^ up
        
        data = [code for code in data if int(code[i]) == int(bit)]
        
        if len(data) == 1:
            break
        
    return str_bit_to_b_10(data[0])


In [ ]:
o2, co2 =parse(data, True), parse(data, False)

In [ ]:
answer_b = o2 * co2
answer_b

In [ ]:
puzzle.answer_b = answer_b

# Day 4

In [ ]:
puzzle = Puzzle(2021, 4)

In [ ]:
data = puzzle.input_data

## Part 1 

In [ ]:
class BoardManager:
    def __init__(self, data):
        data = data.split("\n")
        self.numbers = list(map(int, data[0].split(",")))
        self.boards = list(self.generate_boards(data[2:]))

    def generate_boards(self, data):
        board = []
        for line in data:
            if not line:
                yield BingoBoard(board)
                board = []
            else:
                board.append(line)
            
    def start(self, stop=True):
        last_res = None
        for n in self.numbers:
            for b in self.boards:
                if res := b.draw_number(n):
                    if stop:
                        return res
                    
                    last_res = res
        
        return last_res

class BingoBoard:
    def __init__(self, lines: list[str]):
        self.lines = np.array([[int(letter) for letter in line.split(" ") if letter] for line in lines])
        self.marked = np.full(self.lines.shape, True)
        self.win = False
        
    def draw_number(self, number: str):
        if self.win:
            return 0
        
        for i, j in zip(*np.where(self.lines == int(number))):
            self.marked[i, j] = False
            if self.check_win(i, j):
                return self.get_score(i, j)
    
    def check_win(self, i, j):
        self.win = not np.any(self.marked[i, :]) or not np.any(self.marked[:, j])
        return self.win
    
    def get_score(self, i, j):
        return np.sum(self.lines, where=self.marked) * self.lines[i, j]


In [ ]:
b_m = BoardManager(data)
answer_a = b_m.start()
answer_a

In [ ]:
puzzle.answer_a = answer_a

## Part 2

In [ ]:
b_m = BoardManager(data)
res_b = b_m.start(False)
res_b

In [ ]:
puzzle.answer_b = res_b

# Day 5

In [ ]:
puzzle = Puzzle(2021, 5)

In [ ]:
data = puzzle.input_data

In [ ]:
data.split("\n")

## Part 1 

In [ ]:
def get_gen(start, end, length=0):
    if start < end:
        return [pos for pos in range(start, end + 1)]

    if start > end:
        return [pos for pos in range(start, end - 1, -1)]

    return [start for _ in range(length + 1)]

In [ ]:
def coord_gen(d):
    x0, x1, y0, y1 = int(d["x0"]), int(d["x1"]), int(d["y0"]), int(d["y1"])
    
    x_gen = get_gen(x0, x1, abs(y1 - y0))
    y_gen = get_gen(y0, y1, abs(x1 - x0))
    
    return zip(x_gen, y_gen)

In [ ]:
pattern = re.compile(r"(?P<x0>\d{1,3}),(?P<y0>\d{1,3}) -> (?P<x1>\d{1,3}),(?P<y1>\d{1,3})")

def run(part1=True):
    counter = Counter()
    for line in data.split("\n"):
        d = pattern.match(line).groupdict()
        if part1 and d["x0"] != d["x1"] and d["y0"] != d["y1"]:
            continue

        counter.update(coord_gen(d))
    
    return sum(count >= 2 for count in counter.values())

In [ ]:
answer_a = run()
answer_a

In [ ]:
puzzle.answer_a = answer_a

## Part 2

In [ ]:
answer_b = run(False)
answer_b

In [ ]:
puzzle.answer_b = answer_b

# Day 6

In [ ]:
puzzle = Puzzle(2021, 6)

In [ ]:
data = puzzle.input_data

In [ ]:
data

## Part 1 

In [ ]:
def day(fishies):
    birth = fishies.pop(0, 0)
    new_fishies = defaultdict(int)
    for fishi, number in fishies.items():
        new_fishies[fishi - 1] += number
    new_fishies[6] += birth
    new_fishies[8] += birth
    
    return new_fishies

In [ ]:
def go_fishies(nb_day):
    fishies = Counter(map(int, data.split(",")))
    for _ in range(nb_day):
        fishies = day(fishies)
    
    return sum(fishies.values())


In [ ]:
res_a = go_fishies(80)
res_a

In [ ]:
puzzle.answer_a = res_a

## Part 2

In [ ]:
res_b = go_fishies(256)
res_b

In [ ]:
puzzle.answer_b = res_b

# Day 7

In [152]:
puzzle = Puzzle(2021, 7)

In [153]:
data = puzzle.input_data

In [154]:
data = list(map(int, data.split(",")))

## Part 1 

In [ ]:
def cost_1(position, data):
    return sum(abs(number - position) for number in data)

In [ ]:
m = int(median(data))
m

In [ ]:
res_a = cost_1(m, data)
res_a

In [ ]:
puzzle.answer_a = res_a

## Part 2

In [155]:
def comsumption(distance):
    return distance * (distance + 1) // 2

In [156]:
def cost_2(position, data):
    return sum(comsumption(abs(number - position)) for number in data)

In [157]:
m = int(median(data))
len(data) 

1000

In [159]:
res_b = math.inf
for i in range(max(data)):
    cost = cost_2(i, data)
    if cost < res_b:
        res_b = cost
    else:
        print(i)
        break

473


In [161]:
mean(data)

472.531

In [ ]:
res_b

In [ ]:
puzzle.answer_b = res_b